In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model
from keras import losses

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [28]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 300
steps_per_epoch = len(x_train)

In [8]:
def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return -1 * K.sum(y_true * K.log(y_pred))

In [29]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.KLD(y_true, y_pred)
    return loss1 + loss2

In [30]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
#encoded = LSTM(128, return_sequences=True)(encoded)
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
#decoded = LSTM(256, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=custom_loss, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

In [31]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [32]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/300
2352/2352 [==============================] - 45s 19ms/step - loss: 36.7772
Epoch 2/300
2352/2352 [==============================] - 40s 17ms/step - loss: 35.7068
Epoch 3/300
2352/2352 [==============================] - 40s 17ms/step - loss: 35.6346
Epoch 4/300
2352/2352 [==============================] - 40s 17ms/step - loss: 31.1812
Epoch 5/300
2352/2352 [==============================] - 40s 17ms/step - loss: 15.1742
Epoch 6/300
2352/2352 [==============================] - 40s 17ms/step - loss: 13.0284
Epoch 7/300
2352/2352 [==============================] - 41s 17ms/step - loss: 12.4345
Epoch 8/300
2352/2352 [==============================] - 40s 17ms/step - loss: 11.1181
Epoch 9/300
2352/2352 [==============================] - 40s 17ms/step - loss: 10.4062
Epoch 10/300
2352/2352 [==============================] - 41s 17ms/step - loss: 9.9314
Epoch 11/300
2352/2352 [==============================] - 39s 16ms/step - loss: 9.1930
Epoch 12/300
2352/2352 [===================

2352/2352 [==============================] - 40s 17ms/step - loss: 0.3537
Epoch 96/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.4259
Epoch 97/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.4158
Epoch 98/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.3232
Epoch 99/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.3780
Epoch 100/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.4229
Epoch 101/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.4835
Epoch 102/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.4275
Epoch 103/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.3739
Epoch 104/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.3425
Epoch 105/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.3369
Epoch 106/300
2352/2352 [=========================

2352/2352 [==============================] - 40s 17ms/step - loss: 0.1324
Epoch 189/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1285
Epoch 190/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1333
Epoch 191/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1274
Epoch 192/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1295
Epoch 193/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1215
Epoch 194/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1338
Epoch 195/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1381
Epoch 196/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1141
Epoch 197/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1223
Epoch 198/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.1188
Epoch 199/300
2352/2352 [=====================

2352/2352 [==============================] - 41s 18ms/step - loss: 0.0658
Epoch 282/300
2352/2352 [==============================] - 41s 18ms/step - loss: 0.0766
Epoch 283/300
2352/2352 [==============================] - 42s 18ms/step - loss: 0.0753
Epoch 284/300
2352/2352 [==============================] - 41s 18ms/step - loss: 0.0996
Epoch 285/300
2352/2352 [==============================] - 41s 18ms/step - loss: 0.1257
Epoch 286/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.0711
Epoch 287/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.0779
Epoch 288/300
2352/2352 [==============================] - 40s 17ms/step - loss: 0.0845
Epoch 289/300
2352/2352 [==============================] - 39s 17ms/step - loss: 0.0866
Epoch 290/300
2352/2352 [==============================] - 39s 17ms/step - loss: 0.0788
Epoch 291/300
2352/2352 [==============================] - 39s 17ms/step - loss: 0.0774
Epoch 292/300
2352/2352 [=====================

In [33]:
# model save
lstm_autoencoder.save('model/lstm_autoencoder_custom_loss_bfs.h5')

In [38]:
loaded_model = load_model('model/lstm_autoencoder_custom_loss_bfs.h5')

ValueError: Unknown loss function:custom_loss

In [43]:
model_json = lstm_autoencoder.to_json()
with open('model/lstm_autoencoder_custom_loss_bfs.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model/weights_lstm_autoencoder_custom_loss_bfs.h5')

In [44]:
model = model_from_json(open('model/lstm_autoencoder_custom_loss_bfs.json').read())

NameError: name 'model_from_json' is not defined

In [36]:
loaded_model = lstm_autoencoder

In [37]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

1.3042766420199887


In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [20]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [21]:
latent_vector = np.array(latent_vector)